In [1]:
from dolfin import *
import numpy as np

In [2]:
# Create classes for defining parts of the boundaries and the interior
# of the domain
class Left(SubDomain):
    def inside(self, x, on_boundary):
        return x[:,0]== 0.0

class Right(SubDomain):
    def inside(self, x, on_boundary):
        return x[:,0]== 1.0

class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return x[:,1]== 0.0

class Top(SubDomain):
    def inside(self, x, on_boundary):
        return x[:,1]==1.0
    
class Obstacle(SubDomain):
    def inside(self, x, on_boundary):
        return np.logical_and(np.logical_and(x[:, 0] < 1.0 - np.finfo(float).eps,
                              x[:, 0] > 0.2 - np.finfo(float).eps),
                              np.logical_and(x[:, 1] < 0.7 - np.finfo(float).eps,
                              x[:, 1] > 0.5 - np.finfo(float).eps))

In [3]:
# Initialize sub-domain instances
left = Left()
top = Top()
right = Right()
bottom = Bottom()
obstacle = Obstacle()

In [4]:
# Define mesh
mesh = UnitSquareMesh(dolfin.MPI.comm_world,64, 64)

In [5]:
# Initialize mesh function for interior domains
domains = MeshFunction("size_t", mesh, mesh.topology.dim,0)
domains.array()

array([0, 0, 0, ..., 0, 0, 0], dtype=uint64)

In [6]:
obstacle.mark(domains, 1)

In [7]:
# Initialize mesh function for boundary domains
boundaries = MeshFunction("size_t", mesh, mesh.topology.dim-1,0)
left.mark(boundaries, 1)
top.mark(boundaries, 2)
right.mark(boundaries, 3)
bottom.mark(boundaries, 4)

In [8]:
# Define input data
import ufl
a0 = 1.0
a1 = 0.01
x = SpatialCoordinate(mesh)
g_L = ufl.exp(- 10*np.exp(- pow(x[1] - 0.5, 2)))
g_R = 1.0
f = 1.0

NameError: name 'SpatialCoordinate' is not defined

In [11]:
# Define function space and basis functions
V = FunctionSpace(mesh, ("CG", 2))
u = TrialFunction(V)
v = TestFunction(V)

In [12]:
# Define Dirichlet boundary conditions at top and bottom boundaries
bcs = [DirichletBC(V, 5.0, boundaries, 2),
       DirichletBC(V, 0.0, boundaries, 4)]

NotImplementedError: 

In [13]:
# Define new measures associated with the interior domains and
# exterior boundaries
dx = Measure('dx', domain=mesh, subdomain_data=domains)
ds = Measure('ds', domain=mesh, subdomain_data=boundaries)

NameError: name 'Measure' is not defined

In [ ]:
# Define variational form
F = (inner(a0*grad(u), grad(v))*dx(0) + inner(a1*grad(u), grad(v))*dx(1)
     - g_L*v*ds(1) - g_R*v*ds(3)
     - f*v*dx(0) - f*v*dx(1))

In [ ]:
# Separate left and right hand sides of equation
a, L = lhs(F), rhs(F)

In [ ]:
# Solve problem
u = Function(V)
solve(a == L, u, bcs)

In [ ]:
# Evaluate integral of normal gradient over top boundary
n = FacetNormal(mesh)
m1 = dot(grad(u), n)*ds(2)
v1 = assemble(m1)
print("\int grad(u) * n ds(2) = ", v1)

In [ ]:
# Evaluate integral of u over the obstacle
m2 = u*dx(1)
v2 = assemble(m2)
print("\int u dx(1) = ", v2)

In [ ]:
# Plot solution
import matplotlib.pyplot as plt
plt.figure()
plot(u, title="Solution u")

In [ ]:
# Plot solution and gradient
plt.figure()
plot(grad(u), title="Projected grad(u)")

In [ ]:
# Show plots
plt.show()